<a href="https://colab.research.google.com/github/Samiksha0321/ADSL/blob/master/CNN_SST2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install contractions

In [3]:
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

import pandas as pd
import numpy as np
import string
import unicodedata
import re
import contractions


# from nltk.corpus import stopwords
# import nltk
# nltk.download('stopwords')

!pip install contractions
!pip install datasets
!pip install fasttext

from gensim.models.fasttext import FastText
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import seaborn as sns
import matplotlib.pyplot as plt
from datasets import load_dataset
import plotly.express as px


import fasttext
import fasttext.util

from numpy import array
from numpy import asarray
from numpy import zeros


from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer

Version:  2.6.0
Eager mode:  True
Hub version:  0.12.0
GPU is available


In [4]:
## Loading the fast text model from the drive
from google.colab import drive
drive.mount('/content/drive')
ft = fasttext.load_model('/content/drive/MyDrive/FastText/cc.en.300.bin')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
## DataCleaning Method
def expand_contractions(text):
    return contractions.fix(text)

def preprocess_text(document):
        
        # expand contractions    
        document = expand_contractions(document)

        # Remove all the special characters
        document = re.sub(r'\W', ' ', str(document))

        # remove all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

        # Remove single characters from the start
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)

        # Substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I)

        # Removing prefixed 'b'
        document = re.sub(r'^b\s+', '', document)

        document =re.sub('[^a-zA-Z]', ' ', document)
        document = re.sub(' +', ' ', document)

        # Converting to Lowercase
        document = document.lower()

        # Lemmatization
        #tokens = document.split()
        #tokens = [stemmer.lemmatize(word) for word in tokens]
        #tokens = [word for word in tokens if word not in en_stop]
        #tokens = [word for word in tokens if len(word) > 3]

        #preprocessed_text = ' '.join(tokens)

        return document

In [74]:
# Cleaning of Unclean data

def uncleanProcessing(document):
  document = re.sub("([.,!?;()/\-\'\"])", r' \1 ', document)
  document = re.sub('\s{2,}', ' ', document)
  # Substituting multiple spaces with single space
  document = re.sub(r'\s+', ' ', document, flags=re.I)
  return document

In [149]:
#Importing the Dataset
dataset = load_dataset('gpt3mix/sst2')

Using custom data configuration default
Reusing dataset sst2 (/root/.cache/huggingface/datasets/sst2/default/0.0.0/90167692658fa4abca2ffa3ede1a43a71e2bf671078c5c275c64c4231d5a62fa)


  0%|          | 0/3 [00:00<?, ?it/s]

In [150]:
df_train = pd.DataFrame(list(zip(dataset['train']['text'], dataset['train']['label'])),
               columns =['text', 'label'])
df_val = pd.DataFrame(list(zip(dataset['validation']['text'], dataset['validation']['label'])),
               columns =['text', 'label'])
df_test = pd.DataFrame(list(zip(dataset['test']['text'], dataset['test']['label'])),
               columns =['text', 'label'])

In [134]:
##Cleaning the Dataset
df_train['text'] = df_train['text'].apply(uncleanProcessing)
df_test['text'] = df_test['text'].apply(uncleanProcessing)
df_val['text'] = df_val['text'].apply(uncleanProcessing)

In [135]:
#Spliting the Dataset 
x_train = df_train['text']
y_train = df_train['label']
x_val = df_val['text']
y_val = df_val['label']
x_test = df_test['text']
y_test = df_test['label']

In [136]:
# Tokeninzing the words to integers
tokenizer=Tokenizer()#look at tokenizer documentation
tokenizer.fit_on_texts(x_train)

x_train = tokenizer.texts_to_sequences(x_train)
x_val = tokenizer.texts_to_sequences(x_val)
x_test = tokenizer.texts_to_sequences(x_test)

vocab_size=len(tokenizer.word_index)+1

In [137]:
max_len=[]
for sen in x_train:
  max_len.append(len(sen))
max(max_len)

52

In [138]:
maxlen=52

x_train= pad_sequences(x_train,padding='post',maxlen=maxlen)
x_val = pad_sequences(x_val, padding='post', maxlen=maxlen)  
x_test = pad_sequences(x_test,padding= 'post' , maxlen = maxlen)

In [139]:
# Generating the Embedding Matrix for our Data
embedding_matrix = np.random.rand(vocab_size, 300) #Use fasttext embeddings load the model and get the vector for each word. Initialize with np.random
for word, index in tokenizer.word_index.items():
  embedding_vector = ft.get_word_vector(word)
  if embedding_vector is not None:
    embedding_matrix[index] = embedding_vector

In [140]:
embedding_layer=Embedding(vocab_size,300, weights=[embedding_matrix], input_length= maxlen, trainable=False)
model = tf.keras.Sequential()
model.add(embedding_layer)
model.add(tf.keras.layers.Conv1D(512, 5, activation='relu'))
model.add(tf.keras.layers.Conv1D(256, 5, activation='relu'))  
model.add(tf.keras.layers.GlobalMaxPooling1D())
model.add(tf.keras.layers.Dense(128, activation='relu')) 
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 52, 300)           4147800   
_________________________________________________________________
conv1d_16 (Conv1D)           (None, 48, 512)           768512    
_________________________________________________________________
conv1d_17 (Conv1D)           (None, 44, 256)           655616    
_________________________________________________________________
global_max_pooling1d_8 (Glob (None, 256)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 129       
Total params: 5,604,953
Trainable params: 1,457,153
Non-trainable params: 4,147,800
____________________________________

In [141]:
earlystopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_acc', min_delta=0, patience=2, verbose=0,
    mode='auto', baseline=None, restore_best_weights=True)

In [142]:
history = model.fit(x_train, y_train, epochs=15, batch_size=64, validation_data=(x_val, y_val), verbose=1 , callbacks= [earlystopping])

Epoch 1/15
109/109 [==============================] - 3s 21ms/step - loss: 0.6583 - acc: 0.6022 - val_loss: 0.5480 - val_acc: 0.7294
Epoch 2/15
109/109 [==============================] - 2s 19ms/step - loss: 0.5078 - acc: 0.7462 - val_loss: 0.4799 - val_acc: 0.7695
Epoch 3/15
109/109 [==============================] - 2s 19ms/step - loss: 0.4412 - acc: 0.7925 - val_loss: 0.4793 - val_acc: 0.7741
Epoch 4/15
109/109 [==============================] - 2s 19ms/step - loss: 0.3773 - acc: 0.8309 - val_loss: 0.4860 - val_acc: 0.7683
Epoch 5/15
109/109 [==============================] - 2s 19ms/step - loss: 0.3024 - acc: 0.8668 - val_loss: 0.5165 - val_acc: 0.7901
Epoch 6/15
109/109 [==============================] - 2s 19ms/step - loss: 0.2127 - acc: 0.9133 - val_loss: 0.6150 - val_acc: 0.7500
Epoch 7/15
109/109 [==============================] - 2s 18ms/step - loss: 0.1317 - acc: 0.9467 - val_loss: 0.7990 - val_acc: 0.7511


In [143]:
score = model.evaluate(x_test, y_test, verbose=1)
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

57/57 [==============================] - 0s 7ms/step - loss: 0.4987 - acc: 0.7875
Test Score: 0.49870815873146057
Test Accuracy: 0.7874794006347656
